# Modules and Data Import

In [11]:
from glob import glob
import geopandas as gp
import numpy as np
import pandas as pd
from copy import deepcopy
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from IPython.display import display
import random
import os

pd.options.display.max_rows = 100

import warnings
warnings.filterwarnings("ignore")

try:
    if kernel_is_loaded:
        pass
except:
    os.chdir('/'.join(os.getcwd().split('/')[:-1]))
    kernel_is_loaded = True

In [12]:
xlsx = pd.ExcelFile('data_files/new+old/old/crop_profiles.xlsx')
others = pd.read_excel(xlsx, 'Others')

display('raw:', others.head(3), others.shape)

'raw:'

,FID,lat,long,Crop_Nam_1,STATE_NAME,DISTRICT,Oct_2FN,Nov_1FN,Nov_2FN,Dec_1FN,Dec_2FN,Jan_1FN,Jan_2FN,Feb_1Fn,Feb_2Fn,Mar_1Fn,Mar_2Fn,Apr_1Fn,class
0,14043,24.093073,76.470106,2 Mustard,Madhya Pradesh,RAJGARH,149,141,136,139,141,138,131,134,132,57,0,0,o
1,18755,23.991860,76.553376,2 Mustard,Madhya Pradesh,RAJGARH,165,147,135,135,134,133,132,129,123,92,0,0,o
2,4564,24.068228,76.650815,2 Mustard,Madhya Pradesh,RAJGARH,128,117,131,152,154,154,147,141,120,93,0,0,o


(20138, 19)

# Initial Preprocessing

In [13]:
others.drop(['FID', 'Crop_Nam_1', 'STATE_NAME', 'lat', 'long', 'DISTRICT', 'class'], axis=1, inplace=True)

others.columns = ['oct_2f', 'nov_1f', 'nov_2f', 'dec_1f', 'dec_2f', 'jan_1f', 'jan_2f', 'feb_1f', 
                  'feb_2f', 'mar_1f', 'mar_2f', 'apr_1f']

display('cleaned: ', others.head(), others.shape)

'cleaned: '

,oct_2f,nov_1f,nov_2f,dec_1f,dec_2f,jan_1f,jan_2f,feb_1f,feb_2f,mar_1f,mar_2f,apr_1f
0,149,141,136,139,141,138,131,134,132,57,0,0
1,165,147,135,135,134,133,132,129,123,92,0,0
2,128,117,131,152,154,154,147,141,120,93,0,0
3,127,118,118,118,122,124,117,120,116,96,0,0
4,140,118,131,156,156,154,142,133,113,96,0,0


(20138, 12)

In [14]:
others = others.drop_duplicates(ignore_index=True)

In [15]:
others.to_csv(r'data_files/data_share/uncleaned_others.csv', index=False)

# Data Cleaning

In [ ]:
def harvest_avg_impute(row):
    lst = deepcopy(row)
    hrvst_strt_idx = lst.index.get_loc('jan_2f')
    for i in range(len(lst.loc['jan_2f':'mar_2f'])):
        actual_idx = i + hrvst_strt_idx
        if (lst[actual_idx] == 1) or (lst[actual_idx] == 0):
            if lst[actual_idx-1] < 140:
                lst[actual_idx] = (lst[actual_idx-1] + lst[actual_idx+1])/2
    return lst

def dip_impute(row):
    lst = deepcopy(row)
    act_strt_idx = lst.index.get_loc('jan_1f')
    for i in range(len(lst.loc['jan_1f':'jan_1f'])):
        actual_idx = i + act_strt_idx
        if (lst[actual_idx-1] - lst[actual_idx]) >= 20:
            lst[actual_idx] = (lst[actual_idx-1] + lst[actual_idx+1])/2
    return lst

def sowing_period(row):
    sowing_periods = row.loc['oct_2f':'dec_2f'].index
    sowing_periods_NDVI = row.loc['oct_2f':'dec_2f']
    minima = np.argmin(sowing_periods_NDVI)
    ndvi_values = row.loc['oct_2f':'apr_1f']
    i = minima
    while i < len(sowing_periods):
#         print(ndvi_values[idx])
        if ndvi_values[i] in set(np.arange(110, 141)):
            if (ndvi_values[i+1] - ndvi_values[i]) > 5:
                if ((ndvi_values[i+1] - ndvi_values[i+4]) < 30):
                    return sowing_periods[i]
        i += 1
        
    return 'Unknown'

def harvest_period(row):
    sowing_period_idx = row.index.get_loc(row['sowing_period'])
    i = sowing_period_idx + 6
    while i < len(row.loc[:'apr_1f']):
        if row[i] < 140:
            return row.index[i-1]
        i += 1
    return 'Unknown'

def less_than_150_drop(row):
    sp_loc = row.index.get_loc(row['sowing_period'])
    hp_loc = row.index.get_loc(row['harvest_period'])
    if max(row.iloc[sp_loc+1:hp_loc]) < 150:
        return False
    return True

def data_preprocess(data):
    outliers = data[data.loc[:,'oct_2f':'nov_2f'].apply(lambda row:any((i == 1)|(i == 0) for i in row), axis=1)]

    # Rows which have 0s or 1s in the 3 possible sowing periods for data are dropped
    data.drop(outliers.index, inplace=True)

    # Imputing the possible harvest fns with the average of its immediate neighbours
    data = data.apply(harvest_avg_impute, axis=1)
    
    new_outliers = data[data.loc[:,'jan_2f':'apr_1f'].apply(lambda row:any((i == 1)|(i == 0) for i in row), axis=1)]
    outliers = pd.concat([outliers, new_outliers])
    
    # if 0s and 1s still exit in the possible harvest periods, those rows are dropped
    data.drop(new_outliers.index, inplace=True)

    # Imputing the dec_1f, dec_2f and jan_1f fornights with the averages if the dip is not less than 30 from the adjs 
    data = data.apply(dip_impute, axis=1)

    # Sowing period determination
    data['sowing_period'] = data.apply(sowing_period, axis=1)

    new_outliers = data[data.sowing_period == 'Unknown']
    outliers = pd.concat([outliers, new_outliers])
    
    # Dropping the Unknown sp labels
    data.drop(new_outliers.index, inplace=True)

    data['harvest_period'] = data.apply(harvest_period, axis=1)

    new_outliers = data[data.harvest_period == 'Unknown']
    outliers = pd.concat([outliers, new_outliers])
    
    # Dropping the Unknown harvest labels
    data.drop(new_outliers.index, inplace=True)

    new_outliers = data[data.apply(less_than_150_drop, axis=1) == False]
    outliers = pd.concat([outliers, new_outliers])
    
    # Dropping the rows which have max of NDVI values less than 150 for all the values between sp and hp.
    data = data.drop(new_outliers.index)

    # Dropping the duplicates
    data = data.drop_duplicates(ignore_index=True)

    display(data.head(3), data.shape)

    return data, outliers

others, outliers = 